In [1]:
# ✅ 목표 요약
# ★_재배치_배송.csv → 구분 = 배송
# ★_재배치_회수.csv → 구분 = 회수
# 두 데이터를 병합(append)
# 정렬 및 기존 로직 동일 적용

# ✅ 주요 포인트
# 처리 단계	설명
# 배송/회수 구분	파일 읽을 때 "구분" 컬럼 지정
# 방문코드 구분	배송은 A000001, 회수는 B000001부터 생성
# 병합 후 처리	기존 로직 동일 적용 (순번, No, flag, 요약 등)
# 결과 저장	통합 CSV + 통합 요약 엑셀

In [2]:
import pandas as pd
import os

# 1. 두 파일 경로
file_delivery = r"D:\pythondata\★_재배치_배송.csv"
file_return = r"D:\pythondata\★_재배치_회수.csv"

# 2. CSV 읽기 함수
def read_file(path, gubun_value):
    df = pd.read_csv(
        path,
        sep="\t",
        encoding="utf-8-sig",
        dtype={"대여소": str}
    )
    df["구분"] = gubun_value
    return df

# 3. 배송/회수 데이터프레임 생성
df_delivery = read_file(file_delivery, "배송")
df_return = read_file(file_return, "회수")

# 4. 병합
df = pd.concat([df_delivery, df_return], ignore_index=True)

# 5. 날짜+시간 생성
df["날짜+시간"] = pd.to_datetime(df["날짜"] + " " + df["시간"])

# 6. 정렬용 숫자 변환 + 정렬
df["대여소_정렬"] = pd.to_numeric(df["대여소"], errors="coerce")
df = df.sort_values(by=["대여소_정렬", "날짜+시간"]).reset_index(drop=True)

# 7. 방문코드 생성 (30분 초과 or 대여소 변경 시 새 코드)
visit_code = []
visit_index = 1
prefix = {"배송": "A", "회수": "B"}  # 구분별 prefix

visit_code.append(f"{prefix[df.loc[0, '구분']]}{visit_index:06d}")

for i in range(1, len(df)):
    time_diff = df.loc[i, "날짜+시간"] - df.loc[i - 1, "날짜+시간"]
    location_changed = df.loc[i, "대여소"] != df.loc[i - 1, "대여소"]
    type_changed = df.loc[i, "구분"] != df.loc[i - 1, "구분"]
    if time_diff.total_seconds() > 1800 or location_changed or type_changed:
        visit_index += 1
    code_prefix = prefix[df.loc[i, "구분"]]
    visit_code.append(f"{code_prefix}{visit_index:06d}")

df["방문코드"] = visit_code

# 8. 순번 + No 부여 (중복 자전거번호 처리)
unique_pairs = df.drop_duplicates(subset=["방문코드", "자전거번호"]).copy()
unique_pairs = unique_pairs.sort_values(by=["방문코드", "날짜+시간"])
unique_pairs["순번"] = unique_pairs.groupby("방문코드").cumcount() + 1
unique_pairs["No"] = range(1, 1 + len(unique_pairs))

df = pd.merge(
    df.drop(columns=["순번", "No"], errors="ignore"),
    unique_pairs[["방문코드", "자전거번호", "순번", "No"]],
    on=["방문코드", "자전거번호"],
    how="left"
)

# 9. 중복 자전거번호 → flag
dup_mask = df.duplicated(subset=["방문코드", "자전거번호"], keep="first")
df["flag"] = ""
df.loc[dup_mask, "flag"] = "가짜"

# 10. 자전거대수, first/last time
df["자전거대수"] = df.groupby("방문코드")["대여소"].transform("count")
df["first data&time"] = pd.NaT
df["last data&time"] = pd.NaT
df.loc[df["순번"] == 1, "first data&time"] = df["날짜+시간"]
df.loc[df["순번"] == df["자전거대수"], "last data&time"] = df["날짜+시간"]

# 11. 컬럼 순서 조정 (No → 그 외 → 구분 마지막)
cols = df.columns.tolist()
cols = ["No"] + [col for col in cols if col not in ("No", "구분")] + ["구분"]
df = df[cols]

# 12. 요약 테이블
station_info = df.groupby("방문코드")["대여소"].first().reset_index()
record_counts = df.groupby("방문코드")["대여소"].count().reset_index(name="레코드수")
summary = pd.merge(station_info, record_counts, on="방문코드")
summary = summary[["대여소", "방문코드", "레코드수"]]

# 13. 정리 후 저장
df.drop(columns=["대여소_정렬"], inplace=True)

output_csv_path = r"D:\pythondata\5_통합_방문코드_중복_전체데이터.csv"
df.to_csv(output_csv_path, index=False, encoding="utf-8-sig")

output_summary_path = r"D:\pythondata\5_통합_방문코드_중복_요약.xlsx"
with pd.ExcelWriter(output_summary_path, engine="openpyxl") as writer:
    summary.to_excel(writer, sheet_name="요약", index=False)

print("✅ 통합 CSV 저장 완료:", output_csv_path)
print("✅ 통합 요약 엑셀 저장 완료:", output_summary_path)

✅ 통합 CSV 저장 완료: D:\pythondata\5_통합_방문코드_중복_전체데이터.csv
✅ 통합 요약 엑셀 저장 완료: D:\pythondata\5_통합_방문코드_중복_요약.xlsx
